This code calculates manageable, vulnerable, and irreplaceable carbon across Y2Y

## Load packages and initialize GEE

In [1]:
# import packages
import ee
import geemap
import pandas as pd

In [2]:
# authenticate the EE api
ee.Authenticate()

True

In [3]:
# initialize the EE api
ee.Initialize(project='ee-bermane')

## Define GEE datasets

In [4]:
# define EE datasets
# biomass|
agb = ee.ImageCollection("projects/sat-io/open-datasets/ESA/ESA_CCI_AGB")
rsr = ee.Image("projects/ee-bermane/assets/Root_shoot_ratio_Map_Merged")

# soc
soc_0_30cm_olm = ee.Image("projects/ee-bermane/assets/soc_0_30cm_kg_m2_olm")
soc_30_100cm_olm = ee.Image(
    "projects/ee-bermane/assets/soc_30_100cm_kg_m2_olm")
soc_0_30cm_sothe = ee.Image(
    "projects/ee-bermane/assets/McMaster_WWFCanada_soil_carbon30cm_250m_kg-m2_version3")
soc_0_100cm_sothe = ee.Image(
    "projects/ee-bermane/assets/McMaster_WWFCanada_soil_carbon1m_250m_kg-m2_version3")

# peatlands
peat = ee.Image("projects/sat-io/open-datasets/GLOBAL-PEATLAND-DATABASE")

# landcover
lc = ee.Image("USGS/NLCD_RELEASES/2020_REL/NALCMS")

# netflux
netflux = ee.ImageCollection(
    "projects/wri-datalab/gfw-data-lake/net-flux-forest-extent-per-ha-v1-3-2-2001-2023/net-flux-global-forest-extent-per-ha-2001-2023"
)

# vector
y2y = ee.FeatureCollection("projects/ee-bermane/assets/y2y")
y2y_ecoregions = ee.FeatureCollection(
    "projects/ee-bermane/assets/y2y_ecoregions")
y2y_biomes = ee.FeatureCollection("projects/ee-bermane/assets/y2y_biomes")
rra = ee.FeatureCollection("projects/ee-bermane/assets/ross_river_ipca")
y2y_wdpa = ee.FeatureCollection("projects/ee-bermane/assets/PA_y2y_dat_designated_established_area_20250813")
y2y_oecm = ee.FeatureCollection("projects/ee-bermane/assets/WD_OECM_cliptoPA_20250813")
#y2y_ce = ee.FeatureCollection("projects/ee-bermane/assets/PADUS4_1_ConsEas_Area_20250813")

## Create landcover/biome masks

Landcover key:
* 1: "Temperate or sub-polar needleleaf forest",
* 2: "Sub-polar taiga needleleaf forest",
* 3: "Tropical or sub-tropical broadleaf evergreen forest",
* 4: "Tropical or sub-tropical broadleaf deciduous forest",
* 5: "Temperate or sub-polar broadleaf deciduous forest",
* 6: "Mixed forest",
* 7: "Tropical or sub-tropical shrubland",
* 8: "Temperate or sub-polar shrubland",
* 9: "Tropical or sub-tropical grassland",
* 10: "Temperate or sub-polar grassland",
* 11: "Sub-polar or polar shrubland-lichen-moss",
* 12: "Sub-polar or polar grassland-lichen-moss",
* 13: "Sub-polar or polar barren-lichen-moss",
* 14: "Wetland",
* 15: "Cropland",
* 16: "Barren land",
* 17: "Urban and built-up",
* 18: "Water",
* 19: "Snow and ice"

Biome key:
* 5: Temperate Conifer Forests
* 6: Boreal Forests/Taiga
* 8: Temperate Grasslands, Savannas, and Shrublands
* 11: Tundra
* 13: Deserts and Xeric Shrublands

Peat key:
* 1: Peat dominated
* 2: Peat in soil mosaic

In [5]:
# Rasterize y2y_biomes to use for ecosystem classification based on BIOME_ID:
biomes_raster = y2y_biomes.reduceToImage(
    properties=['BIOME_ID'], reducer=ee.Reducer.first())

# TEMPERATE FORESTS --> FORESTRY
# create a mask based on forest lc types
# and not peatlands (1, 2)
# and biomes mask (5, 8, 13)
temperate_forest_mask = (
    lc.gte(1)
    .And(lc.lte(6))
    .And(peat.mask().Not())
    .And(biomes_raster.eq(5)
         .Or(biomes_raster.eq(8))
         .Or(biomes_raster.eq(13)))
)

# NORTHERN FORESTS --> FORESTRY
# create a mask based on forest lc types
# and not peatlands (1, 2)
# and biomes mask (6, 11)
northern_forest_mask = (
    lc.gte(1)
    .And(lc.lte(6))
    .And(peat.mask().Not())
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# TEMPERATE GRASSLANDS AND SHRUBLANDS --> AGRICULTURE
# create a mask based on grasslands and shrublands LC type (8, 10)
# and not peatlands (1, 2)
# and biomes mask (8)
temperate_grass_shrub_mask = (
    (lc.eq(8)
     .Or(lc.eq(10)))
    .And(peat.mask().Not())
    .And(biomes_raster.eq(8))
)

# MONTANE GRASSLANDS AND SHRUBLANDS --> AGRICULTURE
# create a mask based on grasslands and shrublands LC type (8, 10)
# and not peatlands (1, 2)
# and biomes mask (5)
montane_grass_shrub_mask = (
    (lc.eq(8)
     .Or(lc.eq(10)))
    .And(peat.mask().Not())
    .And(biomes_raster.eq(5))
)

# NORTHERN GRASSLANDS AND SHRUBLANDS --> MINING/OIL/GAS/DEV
# create a mask based on grasslands, shrublands, lichen-moss LC type (8, 10, 11, 12, 13)
# and not peatlands (1, 2)
# and biomes mask (6, 11)
northern_grass_shrub_mask = (
    (lc.eq(8)
     .Or(lc.eq(10))
     .Or(lc.eq(11))
     .Or(lc.eq(12))
     .Or(lc.eq(13)))
    .And(peat.mask().Not())
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# TEMPERATE WETLANDS --> AGRICULTURE
# create a mask based on wetlands LC type (14)
# and not peatlands (1, 2)
# and biomes mask (5, 8)
temperate_wetlands_mask = (
    lc.eq(14)
    .And(peat.mask().Not())
    .And(biomes_raster.eq(5)
         .Or(biomes_raster.eq(8)))
)

# NORTHERN WETLANDS --> MINING/OIL/GAS/DEV
# create a mask based on wetlands LC type (14)
# and not peatlands (1, 2)
# and biomes mask (6, 11)
northern_wetlands_mask = (
    lc.eq(14)
    .And(peat.mask().Not())
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# TEMPERATE PEATLANDS --> AGRICULTURE
# create a mask based on peatlands LC type (1, 2)
# and biomes mask (5, 8)
temperate_peatlands_mask = (
    peat.mask()
    .And(biomes_raster.eq(5)
         .Or(biomes_raster.eq(8)))
)

# NORTHERN PEATLANDS --> MINING/OIL/GAS/DEV
# create a mask based on peatlands LC type (1, 2)
# and biomes mask (6, 11)
northern_peatlands_mask = (
    peat.mask()
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# TEMPERATE BARREN LANDS --> MINING/OIL/GAS/DEV
# LC type (16)
# and not peatlands (1, 2)
# and biomes mask (5, 8)
temperate_barren_mask = (
    lc.eq(16)
    .And(peat.mask().Not())
    .And(biomes_raster.eq(5)
         .Or(biomes_raster.eq(8)))
)

# NORTHERN BARREN LANDS --> MINING/OIL/GAS/DEV
# create a mask based on barren lands LC type (16)
# and not peatlands (1, 2)
# and biomes mask (6, 11)
northern_barren_mask = (
    lc.eq(16)
    .And(peat.mask().Not())
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# check masks to make sure they encompass full extent
mask_check = (
    temperate_forest_mask
    .Or(northern_forest_mask)
    .Or(temperate_grass_shrub_mask)
    .Or(montane_grass_shrub_mask)
    .Or(northern_grass_shrub_mask)
    .Or(temperate_wetlands_mask)
    .Or(northern_wetlands_mask)
    .Or(temperate_peatlands_mask)
    .Or(northern_peatlands_mask)
    .Or(temperate_barren_mask)
    .Or(northern_barren_mask)
)

# Non-manageable systems
# DESERT --> NOT MANAGEABLE
# biomes mask (13)
# and not peatlands (1, 2)
# and not forest (1-6)
desert_mask = (
    lc.gte(7)
    .And(peat.mask().Not())
    .And(biomes_raster.eq(13))
)

# Vegetation cover for biomass replaceability in peatlands
# TEMPERATE FORESTS --> FORESTRY
# create a mask based on forest lc types
# and biomes mask (5, 8, 13)
temperate_forest_bio_mask = (
    lc.gte(1)
    .And(lc.lte(6))
    .And(biomes_raster.eq(5)
         .Or(biomes_raster.eq(8))
         .Or(biomes_raster.eq(13)))
)

# NORTHERN FORESTS --> FORESTRY
# create a mask based on forest lc types
# and biomes mask (6, 11)
northern_forest_bio_mask = (
    lc.gte(1)
    .And(lc.lte(6))
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# not masking out protected areas
# # create a mask out of wdpa layer
# # to remove already protected areas from manageable carbon
# wdpa_mask = ee.Image.constant(1).clip(y2y_wdpa).mask()

## Calculate manageable biomass stock from ESA CCI data

Includes all biomass except for non-forest pixels within the desert biome and the following LC types: barren ground, snow/ice, water, cropland, urban and built-up.

In [6]:
# calc 2021 ESA CCI biomass
# grab the 2021 AGB images
agb_2021 = agb.filter(ee.Filter.stringContains("system:index", "2021")).first()

# create image for litter using Harris ratio (4%)
# leave out dead wood since it is likely sensed already
# mask litter only for forested LC types
# 1: "#033e00",  # Temperate or sub-polar needleleaf forest
# 2: "#939b71",  # Sub-polar taiga needleleaf forest
# 3: "#196d12",  # Tropical or sub-tropical broadleaf evergreen forest
# 4: "#1fab01",  # Tropical or sub-tropical broadleaf deciduous forest
# 5: "#5b725c",  # Temperate or sub-polar broadleaf deciduous forest
# 6: "#6b7d2c",  # Mixed forest

litter_mask = lc.gte(1).And(lc.lte(6))

litter = agb_2021.select(['AGB']).multiply(0.04).multiply(litter_mask)

# create image for BGB (using global rsr map)
bgb = agb_2021.select(['AGB']).multiply(rsr).rename('BGB')

# add AGB + litter and BGB together
bio_2021 = agb_2021.select(['AGB']).add(litter).addBands(
    bgb).rename(['agb_t_ha', 'bgb_t_ha'])

# multiply values by 0.47 to get carbon density
# 0.47 used by Harris et al. (2021)
# mask cropland (15), urban and built-up (17), snow/ice (19), water (18)
bio_2021 = bio_2021.multiply(0.47).updateMask(
    lc.neq(15).And(lc.neq(17)).And(lc.neq(18)).And(lc.neq(19)))

# mask desert pixels from bio_2021
bio_2021 = (
    bio_2021
    .updateMask(desert_mask.Not())
)

# compute per-pixel area in ha
pixel_area_ha = ee.Image.pixelArea().divide(10000)

# Create carbon layer mask to filter pixel area raster
carbon_mask = bio_2021.select(['agb_t_ha']).neq(0)

# Mask pixel_area_ha to carbon layers
pixel_area_agb_extent = pixel_area_ha.updateMask(carbon_mask)

# calculate biomass stock
bio_2021_stock = (
    bio_2021
    .multiply(pixel_area_ha)
    .rename(['agb_t', 'bgb_t'])
)

# combine bands for manageable biomass density and stock
manageable_bio_density = (
    bio_2021
    .select(['agb_t_ha', 'bgb_t_ha'])
    .reduce(ee.Reducer.sum())
    .rename('manageable_biomass_t_ha')
)

manageable_bio_stock = (
    bio_2021_stock
    .select(['agb_t', 'bgb_t'])
    .reduce(ee.Reducer.sum())
    .rename('manageable_biomass_t')
)

## Calculate manageable soil carbon stock from Sothe Canada data and Open Land Map US data

Includes all SOC 0-30cm and 30-100cm except for non-forest pixels within the desert biome, and the following LC types: snow/ice, water, cropland, urban and built-up.

Sets up two scenarios in areas prone to mining/oil/gas development: 0-30cm manageable SOC accounting for seismic line disturbances and 0-100cm manageable SOC for open pit mining disturbances

In [7]:
# calc open land map soc for 0-30cm and 30-100cm
# multiply by 10 to get t/ha
# mask water/snow/ice/cropland, urban and built-up
soc_0_30cm_olm_t_ha = soc_0_30cm_olm.multiply(10).rename(
    'soc_0_30cm_t_ha')

soc_30_100cm_olm_t_ha = soc_30_100cm_olm.multiply(10).rename(
    'soc_30_100cm_t_ha')

# add both to same image
soc_olm_t_ha = soc_0_30cm_olm_t_ha.addBands(soc_30_100cm_olm_t_ha)

# resample to match sothe to blend images
soc_olm_t_ha_reproj_sothe = soc_olm_t_ha.resample('bilinear').toFloat()

In [8]:
# calc sothe soc for 0-30cm and 30-100cm
# multiply by 10 to get t/ha
# mask water/snow/ice/cropland, urban and built-up
soc_0_30cm_sothe_t_ha = soc_0_30cm_sothe.multiply(10).rename(
    'soc_0_30cm_t_ha')

soc_0_100cm_sothe_t_ha = soc_0_100cm_sothe.multiply(10).rename(
    'soc_0_100cm_t_ha')

soc_30_100cm_sothe_t_ha = soc_0_100cm_sothe_t_ha.subtract(soc_0_30cm_sothe_t_ha).rename(
    'soc_30_100cm_t_ha')

# add both to same image
soc_sothe_t_ha = soc_0_30cm_sothe_t_ha.addBands(soc_30_100cm_sothe_t_ha)

# blend sothe and olm carbon across y2y
# band names must match to mosaic!
# reproject to original scale
soc_blend = ee.ImageCollection([soc_olm_t_ha_reproj_sothe, soc_sothe_t_ha]).mosaic().reproject(
    crs=soc_sothe_t_ha.projection(),
    crsTransform=soc_sothe_t_ha.projection().getInfo().get('transform')
    # mask snow/ice, water, cropland, urban and built-up
).updateMask(lc.neq(18).And(lc.neq(19)).And(lc.neq(15)).And(lc.neq(17)))

# mask desert from soc_blend
soc_blend = (
    soc_blend
    .updateMask(desert_mask.Not())
)

# # Create soc layer mask to filter pixel area raster
# # use 0-30cm soc band since masking more pixels out of 30-100cm band
# soc_mask = soc_blend.select('soc_0_30cm_t_ha').mask().neq(0)

# # Mask pixel_area_ha to carbon layers
# pixel_area_soc_extent = pixel_area_ha.updateMask(soc_mask)

# # multiply by pixel area to get total carbon per pixel
# soc_blend_stock = soc_blend.multiply(pixel_area_ha).rename('soc_t').addBands(
#     pixel_area_soc_extent.rename(['pixel_area_soc_extent_ha'])
# ).addBands(
#     pixel_area_ha.rename(['pixel_area_ha'])
# )

# calculate soc stock
soc_blend_stock = (
    soc_blend
    .multiply(pixel_area_ha)
    .rename('soc_0_30cm_t', 'soc_30_100cm_t')
)

# combine SOC bands for manageable soc density and stock
manageable_soc_density = (
    soc_blend
    .select(['soc_0_30cm_t_ha', 'soc_30_100cm_t_ha'])
    .reduce(ee.Reducer.sum())
    .rename('manageable_soc_t_ha')
)

manageable_soc_stock = (
    soc_blend_stock
    .select(['soc_0_30cm_t', 'soc_30_100cm_t'])
    .reduce(ee.Reducer.sum())
    .rename('manageable_soc_t')
)

## Calculate vulnerable biomass carbon stock
All manageable biomass carbon is vulnerable, so no calculations needed.

In [9]:
# rename total vulnerable biomass carbon density and stock
vulnerable_bio_density = manageable_bio_density.rename(
    'vulnerable_biomass_t_ha')

vulnerable_bio_stock = manageable_bio_stock.rename('vulnerable_biomass_t')

## Calculate vulnerable SOC stock
Using two scenarios for regions where mining/oil/gas development are biggest threat

In [10]:
# first we need to split manageable soc density into layers for the two scenarios:
# seismic lines and mining

# mask soc_blend 30-100cm band for 0-30cm SOC scenario
updated_30_100cm_soc_sl = (
    soc_blend
    .select('soc_30_100cm_t_ha')
    .updateMask(temperate_forest_mask.Not())
    .updateMask(northern_forest_mask.Not())
    .updateMask(temperate_grass_shrub_mask.Not())
    .updateMask(montane_grass_shrub_mask.Not())
    .updateMask(northern_grass_shrub_mask.Not())
    .updateMask(northern_wetlands_mask.Not())
    .updateMask(northern_peatlands_mask.Not())
    .updateMask(temperate_barren_mask.Not())
    .updateMask(northern_barren_mask.Not())
)

# mask soc_blend density 30-100cm band for 0-100cm SOC scenario
updated_30_100cm_soc_m = (
    soc_blend
    .select('soc_30_100cm_t_ha')
    .updateMask(temperate_forest_mask.Not())
    .updateMask(northern_forest_mask.Not())
    .updateMask(temperate_grass_shrub_mask.Not())
    .updateMask(montane_grass_shrub_mask.Not())
)

# split soc_blend density into two scenarios: seismic lines (sl) and mining (m)
soc_blend_sl = soc_blend.addBands(updated_30_100cm_soc_sl, overwrite=True)
soc_blend_m = soc_blend.addBands(updated_30_100cm_soc_m, overwrite=True)

# combine soc_blend density bands for two scenarios
soc_blend_sl = (
    soc_blend_sl
    .select(['soc_0_30cm_t_ha', 'soc_30_100cm_t_ha'])
    .reduce(ee.Reducer.sum())
    .rename('soc_sl_t_ha')
)

soc_blend_m = (
    soc_blend_m
    .select(['soc_0_30cm_t_ha', 'soc_30_100cm_t_ha'])
    .reduce(ee.Reducer.sum())
    .rename('soc_m_t_ha')
)

# seismic lines scenario
# set vulnerable SOC based on different masks
# temperate_forest_mask --> 0%
# northern_forest_mask --> 0%
# temperate_grass_shrub_mask --> 36.1%
# montane_grass_shrub_mask --> 36.1%
# northern_grass_shrub_mask --> 32.2%
# temperate_wetlands_mask --> 42%
# northern_wetlands_mask --> 32.2%
# temperate_peatlands_mask --> 100% or 135 t/ha
# northern_peatlands_mask --> 32.2%
# temperate_barren_mask --> 32.2%
# northern_barren_mask --> 32.2%
vulnerable_soc_sl_density = (
    soc_blend_sl
    .where(temperate_forest_mask, 0)
    .where(northern_forest_mask, 0)
    .where(temperate_grass_shrub_mask, soc_blend_sl.multiply(.361))
    .where(montane_grass_shrub_mask, soc_blend_sl.multiply(.361))
    .where(northern_grass_shrub_mask, soc_blend_sl.multiply(.322))
    .where(temperate_wetlands_mask, soc_blend_sl.multiply(.42))
    .where(northern_wetlands_mask, soc_blend_sl.multiply(.322))
    .where(temperate_peatlands_mask.And(soc_blend_sl.gte(135)), 135)
    .where(northern_peatlands_mask, soc_blend_sl.multiply(.322))
    .where(temperate_barren_mask, soc_blend_sl.multiply(.322))
    .where(northern_barren_mask, soc_blend_sl.multiply(.322))
    .rename('vulnerable_soc_sl_t_ha')
)

# mining scenario
# set vulnerable SOC based on different masks
# temperate_forest_mask --> 0%
# northern_forest_mask --> 0%
# temperate_grass_shrub_mask --> 36.1%
# montane_grass_shrub_mask --> 36.1%
# northern_grass_shrub_mask --> 91%
# temperate_wetlands_mask --> 42%
# northern_wetlands_mask --> 91%
# temperate_peatlands_mask --> 100% or 135 t/ha
# northern_peatlands_mask --> 91%
# temperate_barren_mask --> 91%
# northern_barren_mask --> 91%
vulnerable_soc_m_density = (
    soc_blend_m
    .where(temperate_forest_mask, 0)
    .where(northern_forest_mask, 0)
    .where(temperate_grass_shrub_mask, soc_blend_m.multiply(.361))
    .where(montane_grass_shrub_mask, soc_blend_m.multiply(.361))
    .where(northern_grass_shrub_mask, soc_blend_m.multiply(.91))
    .where(temperate_wetlands_mask, soc_blend_m.multiply(.42))
    .where(northern_wetlands_mask, soc_blend_m.multiply(.91))
    .where(temperate_peatlands_mask.And(soc_blend_m.gte(135)), 135)
    .where(northern_peatlands_mask, soc_blend_m.multiply(.91))
    .where(temperate_barren_mask, soc_blend_m.multiply(.91))
    .where(northern_barren_mask, soc_blend_m.multiply(.91))
    .rename('vulnerable_soc_m_t_ha')
)

# calculate vulnerable soc stock for both scenarios
vulnerable_soc_sl_stock = (
    vulnerable_soc_sl_density
    .multiply(pixel_area_ha)
    .rename('vulnerable_soc_sl_t')
)

vulnerable_soc_m_stock = (
    vulnerable_soc_m_density
    .multiply(pixel_area_ha)
    .rename('vulnerable_soc_m_t')
)

## Calculate recoverable biomass carbon stock
Based on 30 year recovery.

In [11]:
# set recoverable biomass C based on different masks
# temperate_forest_bio_mask --> 1.658*20 + 2.127*10 t/ha
# northern_forest_bio_mask --> 1.268*20 + 0.655*10 t/ha
# use bio_mask because we don't mask peat here
# everywhere else is just the same amount as vulnerable
recoverable_bio_density = (
    vulnerable_bio_density
    .where(temperate_forest_bio_mask, 1.658*20 + 2.127*10)
    .where(northern_forest_bio_mask, 1.268*20 + 0.655*10)
    .rename('recoverable_biomass_t_ha')
)

# set recoverable value ceiling at vulnerable values
recoverable_bio_density = recoverable_bio_density.where(
    recoverable_bio_density.gt(vulnerable_bio_density), vulnerable_bio_density)

# calculate recoverable biomass stock
recoverable_bio_stock = (
    recoverable_bio_density
    .multiply(pixel_area_ha)
    .rename('recoverable_biomass_t')
)

## Calculate recoverable SOC stock

Recovery rates are same in both seismic lines and mining scenario

In [12]:
# calculate remaining soc density after land conversion for both scenarios
remaining_soc_sl_density = (
    soc_blend_sl
    .subtract(vulnerable_soc_sl_density)
    .rename('remaining_soc_sl_t_ha')
)

remaining_soc_m_density = (
    soc_blend_m
    .subtract(vulnerable_soc_m_density)
    .rename('remaining_soc_m_t_ha')
)

# remaining_soc_sl_stock = (
#     remaining_soc_sl_density
#     .multiply(pixel_area_ha)
#     .rename('remaining_soc_sl_t')
# )

# remaining_soc_m_stock = (
#     remaining_soc_m_density
#     .multiply(pixel_area_ha)
#     .rename('remaining_soc_m_t')
# )

# set recoverable SOC based on different masks
# temperate_forest_mask --> none vulnerable
# northern_forest_mask --> none vulnerable
# temperate_grass_shrub_mask --> 57.3% of remaining added back
# montane_grass_shrub_mask --> 25% of remaining added back
# northern_grass_shrub_mask --> 1.3*30 t/ha
# temperate_wetlands_mask --> 1.226*30 t/ha
# northern_wetlands_mask --> 1.226*30 t/ha
# temperate_peatlands_mask --> 0.28*30 t/ha
# northern_peatlands_mask --> 0.21*30 t/ha
# temperate_barren_mask --> 1.3*30 t/ha
# northern_barren_mask --> 1.3*30 t/ha
recoverable_soc_sl_density = (
    remaining_soc_sl_density
    .where(temperate_forest_mask, 0)
    .where(northern_forest_mask, 0)
    .where(temperate_grass_shrub_mask, remaining_soc_sl_density.multiply(.573))
    .where(montane_grass_shrub_mask, remaining_soc_sl_density.multiply(.25))
    .where(northern_grass_shrub_mask, 1.3*30)
    .where(temperate_wetlands_mask, 1.226*30)
    .where(northern_wetlands_mask, 1.226*30)
    .where(temperate_peatlands_mask, 0.28*30)
    .where(northern_peatlands_mask, 0.21*30)
    .where(temperate_barren_mask, 1.3*30)
    .where(northern_barren_mask, 1.3*30)
    .rename('recoverable_soc_sl_t_ha')
)

recoverable_soc_m_density = (
    remaining_soc_m_density
    .where(temperate_forest_mask, 0)
    .where(northern_forest_mask, 0)
    .where(temperate_grass_shrub_mask, remaining_soc_m_density.multiply(.573))
    .where(montane_grass_shrub_mask, remaining_soc_m_density.multiply(.25))
    .where(northern_grass_shrub_mask, 1.3*30)
    .where(temperate_wetlands_mask, 1.226*30)
    .where(northern_wetlands_mask, 1.226*30)
    .where(temperate_peatlands_mask, 0.28*30)
    .where(northern_peatlands_mask, 0.21*30)
    .where(temperate_barren_mask, 1.3*30)
    .where(northern_barren_mask, 1.3*30)
    .rename('recoverable_soc_sl_t_ha')
)

# set recoverable value ceiling at vulnerable values
recoverable_soc_sl_density = recoverable_soc_sl_density.where(
    recoverable_soc_sl_density.gt(vulnerable_soc_sl_density), vulnerable_soc_sl_density)

recoverable_soc_m_density = recoverable_soc_m_density.where(
    recoverable_soc_m_density.gt(vulnerable_soc_m_density), vulnerable_soc_m_density)

# # calculate recoverable soc stock
# recoverable_soc_sl_stock = (
#     recoverable_soc_sl_density
#     .multiply(pixel_area_ha)
#     .rename('recoverable_soc_sl_t')
# )

# recoverable_soc_m_stock = (
#     recoverable_soc_m_density
#     .multiply(pixel_area_ha)
#     .rename('recoverable_soc_sl_t')
# )

## Calculate Irrecoverable biomass C and SOC
Vulnerable minus recoverable

In [13]:
# Irrecoverable biomass C density
irrecoverable_bio_density = (
    vulnerable_bio_density
    .subtract(recoverable_bio_density)
    .rename('irrecoverable_biomass_t_ha')
)

# calculate irrecoverable biomass stock
irrecoverable_bio_stock = (
    irrecoverable_bio_density
    .multiply(pixel_area_ha)
    .rename('irrecoverable_biomass_t')
)

# irrecoverable SOC density
irrecoverable_soc_sl_density = (
    vulnerable_soc_sl_density
    .subtract(recoverable_soc_sl_density)
    .rename('irrecoverable_soc_sl_t_ha')
)

irrecoverable_soc_m_density = (
    vulnerable_soc_m_density
    .subtract(recoverable_soc_m_density)
    .rename('irrecoverable_soc_m_t_ha')
)

# calculate irrecoverable SOC stock
irrecoverable_soc_sl_stock = (
    irrecoverable_soc_sl_density
    .multiply(pixel_area_ha)
    .rename('irrecoverable_soc_sl_t')
)

irrecoverable_soc_m_stock = (
    irrecoverable_soc_m_density
    .multiply(pixel_area_ha)
    .rename('irrecoverable_soc_m_t')
)

# combine irrecoverable biomass and SOC density
irrecoverable_combined_sl_density = (
    irrecoverable_soc_sl_density
    .addBands(irrecoverable_bio_density)
    .reduce(ee.Reducer.sum())
    .rename('irrecoverable_combined_sl_t_ha')
)

irrecoverable_combined_m_density = (
    irrecoverable_soc_m_density
    .addBands(irrecoverable_bio_density)
    .reduce(ee.Reducer.sum())
    .rename('irrecoverable_combined_m_t_ha')
)

# calculate irrecoverable combined stock
irrecoverable_combined_sl_stock = (
    irrecoverable_combined_sl_density
    .multiply(pixel_area_ha)
    .rename('irrecoverable_combined_sl_t')
)

irrecoverable_combined_m_stock = (
    irrecoverable_combined_m_density
    .multiply(pixel_area_ha)
    .rename('irrecoverable_combined_m_t')
)

## Calculate Hotspots

In [14]:
# create function to mask hotspots
def mask_hotspots(image, layer_name, percentile, features, gt):
    # calculate percentile
    perc = image.reduceRegion(
        reducer=ee.Reducer.percentile([percentile]),
        geometry=features.geometry(),
        crs=image.projection().getInfo().get('crs'),
        crsTransform=image.projection().getInfo().get('transform'),
        maxPixels=1e30).get(layer_name)

    # apply percentile mask to image
    # greater than
    if gt == 'gt':
        image_out = image.updateMask(image.gt(ee.Number(perc)))
    # less than
    if gt == 'lte':
        image_out = image.updateMask(image.lte(ee.Number(perc)))

    # return masked image
    return (image_out)

In [15]:
# create layer with top 20% of biomass density
irrecoverable_bio_density_top20 = mask_hotspots(irrecoverable_bio_density
                                                .updateMask(irrecoverable_bio_density.gt(0)),
                                                       'irrecoverable_biomass_t_ha',
                                                       80,
                                                       y2y,
                                                       'gt')

# # create layer with top 5% of biomass density
# irrecoverable_bio_density_top5 = mask_hotspots(irrecoverable_bio_density,
#                                                'irrecoverable_biomass_t_ha',
#                                                95,
#                                                y2y,
#                                                'gt')

# # create layer with top 1% of biomass density
# irrecoverable_bio_density_top1 = mask_hotspots(irrecoverable_bio_density,
#                                                'irrecoverable_biomass_t_ha',
#                                                99,
#                                                y2y,
#                                                'gt')

# create layer with top 20% of soc density
irrecoverable_soc_sl_density_top20 = mask_hotspots(irrecoverable_soc_sl_density
                                                   .updateMask(irrecoverable_soc_sl_density.gt(0)),
                                                  'irrecoverable_soc_sl_t_ha',
                                                  80,
                                                  y2y,
                                                  'gt')

irrecoverable_soc_m_density_top20 = mask_hotspots(irrecoverable_soc_m_density
                                                  .updateMask(irrecoverable_soc_m_density.gt(0)),
                                                 'irrecoverable_soc_m_t_ha',
                                                 80,
                                                 y2y,
                                                 'gt')

# # create layer with top 5% of soc density
# irrecoverable_soc_sl_density_top5 = mask_hotspots(irrecoverable_soc_sl_density,
#                                                   'irrecoverable_soc_sl_t_ha',
#                                                   95,
#                                                   y2y,
#                                                   'gt')

# irrecoverable_soc_m_density_top5 = mask_hotspots(irrecoverable_soc_m_density,
#                                                  'irrecoverable_soc_m_t_ha',
#                                                  95,
#                                                  y2y,
#                                                  'gt')

# # create layer with top 1% of soc density
# irrecoverable_soc_sl_density_top1 = mask_hotspots(irrecoverable_soc_sl_density,
#                                                   'irrecoverable_soc_sl_t_ha',
#                                                   99,
#                                                   y2y,
#                                                   'gt')

# irrecoverable_soc_m_density_top1 = mask_hotspots(irrecoverable_soc_m_density,
#                                                  'irrecoverable_soc_m_t_ha',
#                                                  99,
#                                                  y2y,
#                                                  'gt')

# # create layer with top 20% of combined density
# irrecoverable_combined_sl_density_top20 = mask_hotspots(irrecoverable_combined_sl_density,
#                                                        'irrecoverable_combined_sl_t_ha',
#                                                        80,
#                                                        y2y,
#                                                        'gt')

# irrecoverable_combined_m_density_top20 = mask_hotspots(irrecoverable_combined_m_density,
#                                                       'irrecoverable_combined_m_t_ha',
#                                                       80,
#                                                       y2y,
#                                                       'gt')

# # create layer with top 5% of combined density
# irrecoverable_combined_sl_density_top5 = mask_hotspots(irrecoverable_combined_sl_density,
#                                                        'irrecoverable_combined_sl_t_ha',
#                                                        95,
#                                                        y2y,
#                                                        'gt')

# irrecoverable_combined_m_density_top5 = mask_hotspots(irrecoverable_combined_m_density,
#                                                       'irrecoverable_combined_m_t_ha',
#                                                       95,
#                                                       y2y,
#                                                       'gt')

# # create layer with top 1% of combined density
# irrecoverable_combined_sl_density_top1 = mask_hotspots(irrecoverable_combined_sl_density,
#                                                        'irrecoverable_combined_sl_t_ha',
#                                                        99,
#                                                        y2y,
#                                                        'gt')

# irrecoverable_combined_m_density_top1 = mask_hotspots(irrecoverable_combined_m_density,
#                                                       'irrecoverable_combined_m_t_ha',
#                                                       99,
#                                                       y2y,
#                                                       'gt')

In [16]:
# calculate hotspots for netflux
# first mosaic image
# and reproject to 100 m since all the flux output layers are going to be at 100m
# clip to y2y
# then convert to carbon, and divide by 23 to get annual value
netflux_c = (
    netflux
    .mosaic()
    .reproject(crs=netflux.first().projection().getInfo().get('crs'),
               scale=100)
    .clip(y2y)
    .multiply(12/44)
    .divide(23)
    .rename('netflux_c_t_ha')
)

# update mask to remove emissions values
# set sequestration to positive
# reproject to match carbon
netflux_c = netflux_c.updateMask(netflux_c.lt(0)).multiply(-1)

# create layer with top 20% of net removals
# need to clip to y2y to only get hotspots in y2y
netflux_c_top20 = mask_hotspots(netflux_c,
                               'netflux_c_t_ha',
                               80,
                               y2y,
                               'gt')

# create layer with top 5% of net removals
# need to clip to y2y to only get hotspots in y2y
netflux_c_top5 = mask_hotspots(netflux_c,
                               'netflux_c_t_ha',
                               95,
                               y2y,
                               'gt')

# create layer with top 1% of net removals
# need to clip to y2y to only get hotspots in y2y
netflux_c_top1 = mask_hotspots(netflux_c,
                               'netflux_c_t_ha',
                               99,
                               y2y,
                               'gt')

# load export crs info
flux_out_proj = netflux.first().projection().getInfo().get('crs')
flux_out_scale = netflux.first().projection().nominalScale().getInfo()

In [17]:
# create raster with overlapping hotspots of irrecoverable carbon and netflux
# top 20% of values

irrecoverable_bio_netflux_top20 = (
    irrecoverable_bio_density_top20
    .where(irrecoverable_bio_density_top20.gt(0), 1)
    .addBands(netflux_c_top20.where(netflux_c_top20.gt(0), 2))
    .reduce(ee.Reducer.sum())
    .rename('irrecoverable_netflux_hotspots')
)

# irrecoverable_sl_netflux_top20 = (
#     netflux_c_top20
#     .where(netflux_c_top20.gt(0), 2)
#     .addBands(irrecoverable_combined_sl_density_top20
#               .where(irrecoverable_combined_sl_density_top20.gt(0), 1))
#     .reduce(ee.Reducer.sum())
#     .rename('irrecoverable_netflux_hotspots')
# )

# irrecoverable_m_netflux_top20 = (
#     netflux_c_top20
#     .where(netflux_c_top20.gt(0), 2)
#     .addBands(irrecoverable_combined_m_density_top20
#               .where(irrecoverable_combined_m_density_top20.gt(0), 1))
#     .reduce(ee.Reducer.sum())
#     .rename('irrecoverable_netflux_hotspots')
# )

In [18]:
# run bivariate analysis on irrecoverable carbon and netflux

# function to normalize and bin raster
def normalize_bin(image):

    # get first band name
    band = image.bandNames().get(0)
    
    # calculate min and max of raster (single band image)
    stats = image.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=y2y.geometry(),
        crs=image.projection().getInfo().get('crs'),
        crsTransform=image.projection().getInfo().get('transform'),
        #scale=scale,
        maxPixels=1e30
        )

    min = ee.Number(stats.get(ee.String(band).cat('_min')))
    max = ee.Number(stats.get(ee.String(band).cat('_max')))

    # normalize raster
    norm = image.unitScale(min, max)

    # compute quantiles for 5 bins
    quantiles = norm.reduceRegion(
        reducer=ee.Reducer.percentile([20, 40, 60, 80]),
        geometry=y2y.geometry(),
        crs=norm.projection(),
        crsTransform=image.projection().getInfo().get('transform'),
        #scale=scale,
        maxPixels=1e30
        )
    
    # extract quantiles
    p20 = ee.Number(quantiles.get(ee.String(band).cat('_p20')))
    p40 = ee.Number(quantiles.get(ee.String(band).cat('_p40')))
    p60 = ee.Number(quantiles.get(ee.String(band).cat('_p60')))
    p80 = ee.Number(quantiles.get(ee.String(band).cat('_p80')))

    # bin raster based on quantiles
    bin = (
        norm
        .where(norm.gt(p20).And(norm.lte(p40)), 2)
        .where(norm.gt(p40).And(norm.lte(p60)), 3)
        .where(norm.gt(p60).And(norm.lte(p80)), 4)
        .where(norm.gt(p80), 5)
        .where(norm.lte(p20), 1)
        )

    # # bin raster based on values
    # bin = (
    #     norm
    #     .where(norm.lte(0.2), 1)
    #     .where(norm.gt(0.2).And(norm.lte(0.4)), 2)
    #     .where(norm.gt(0.4).And(norm.lte(0.6)), 3)
    #     .where(norm.gt(0.6).And(norm.lte(0.8)), 4)
    #     .where(norm.gt(0.8).And(norm.lte(1)), 5)
    # )

    # return binned raster
    return bin

# normalize netflux
netflux_bin = normalize_bin(netflux_c)

# normalize irrecoverable carbon
irrecoverable_combined_sl_density_bin = normalize_bin(irrecoverable_combined_sl_density.updateMask(irrecoverable_combined_sl_density.gt(0)))
irrecoverable_combined_m_density_bin = normalize_bin(irrecoverable_combined_m_density.updateMask(irrecoverable_combined_m_density.gt(0)))
irrecoverable_bio_density_bin = normalize_bin(irrecoverable_bio_density.updateMask(irrecoverable_bio_density.gt(0)))


# calculate bivariate rasters
# consider null values as 0 bin
irrecoverable_bio_netflux_bivar = (
    irrecoverable_bio_density_bin
    .addBands(netflux_bin.multiply(6))
    .reduce(ee.Reducer.sum())
    .rename('irrecoverable_netflux_hotspots')
)

# irrecoverable_combined_sl_netflux_bivar = netflux_bin.multiply(5).add(irrecoverable_combined_sl_density_bin)
# irrecoverable_combined_m_netflux_bivar = netflux_bin.multiply(5).add(irrecoverable_combined_m_density_bin)


## Add layers to map

In [19]:
# set biomass palette
biomass_palette = [
    "#C6ECAE", "#A1D490", "#7CB970", "#57A751", "#348E32",
    "#267A29", "#176520", "#0C4E15", "#07320D", "#031807"
]

# set land cover palette
lc_palette = [
    "#033e00",  # Temperate or sub-polar needleleaf forest
    "#939b71",  # Sub-polar taiga needleleaf forest
    "#196d12",  # Tropical or sub-tropical broadleaf evergreen forest
    "#1fab01",  # Tropical or sub-tropical broadleaf deciduous forest
    "#5b725c",  # Temperate or sub-polar broadleaf deciduous forest
    "#6b7d2c",  # Mixed forest
    "#b29d29",  # Tropical or sub-tropical shrubland
    "#b48833",  # Temperate or sub-polar shrubland
    "#e9da5d",  # Tropical or sub-tropical grassland
    "#e0cd88",  # Temperate or sub-polar grassland
    "#a07451",  # Sub-polar or polar shrubland-lichen-moss
    "#bad292",  # Sub-polar or polar grassland-lichen-moss
    "#3f8970",  # Sub-polar or polar barren-lichen-moss
    "#6ca289",  # Wetland
    "#e6ad6a",  # Cropland
    "#a9abae",  # Barren land
    "#db2126",  # Urban and built-up
    "#4c73a1",  # Water
    "#fff7fe",  # Snow and ice
]

bivariate_palette = [
  '#E6FFFF', '#B2FFFF', '#80FFFF', '#4DFFFF', '#00FFFF',
  '#FFE6E6', '#FFB2B2', '#FF8080', '#FF4D4D', '#FF0000',
  '#E6E6FF', '#B2B2FF', '#8080FF', '#4D4DFF', '#0000FF',
  '#FFE6E6', '#FFB2B2', '#FF8080', '#FF4D4D', '#FF0000',
  '#E6FFFF', '#B2FFFF', '#80FFFF', '#4DFFFF', '#00FFFF'
]

In [20]:
# create a map
m = geemap.Map()

# add layers
m.addLayer(y2y, {}, "Y2Y")
# m.addLayer(desert_mask, {}, "Desert Mask")
# m.addLayer(y2y_biomes, {}, "Biomes")
# m.addLayer(lc.clip(y2y), {"min": 1, "max": 19,
#                           "palette": lc_palette}, 'Landcover')
# m.addLayer(agb_2021.select(['AGB']).clip(y2y), {"min": 1, "max": 450,
#            "palette": biomass_palette}, "AGB")
# m.addLayer(bio_2021.select(['agb_t_ha']).clip(y2y), {"min": 1, "max": 450,
#            "palette": biomass_palette}, "AGB_t_ha")
# m.addLayer(pixel_area_agb_extent.clip(y2y), {}, 'Carbon Mask')
# m.addLayer(manageable_bio_stock.select(['agb_t']).clip(y2y), {
#     "min": 1, "max": 100, "palette": biomass_palette}, 'CCI AGB')
# m.addLayer(soc_blend.select(['soc_0_30cm_t_ha']).clip(y2y), {"min": 1, "max": 450,
#            "palette": biomass_palette}, "SOC Density Blend 0-30cm")
# m.addLayer(soc_blend.select(['soc_30_100cm_t_ha']).clip(y2y), {"min": 1, "max": 450,
#            "palette": biomass_palette}, "SOC Density Blend 30-100cm")
# m.addLayer(pixel_area_soc_extent.clip(y2y), {}, 'Carbon Mask')
# m.addLayer(peat.clip(y2y), {}, "Peat Distribution")
# m.addLayer(rra, {}, "RRA")
# m.addLayer(vulnerable_soc_dens.select('soc_0_30cm_t_ha').clip(y2y), {
#            "min": 0, "max": 450, "palette": biomass_palette}, "Vulnerable SOC Density 0-30cm")
# m.addLayer(vulnerable_soc_dens.select('soc_30_100cm_t_ha').clip(y2y), {
#            "min": 0, "max": 450, "palette": biomass_palette}, "Vulnerable SOC Density 30-100cm")

# final stock layers
# m.addLayer(manageable_bio_density.clip(y2y), {
#            "min": 0, "max": 450, "palette": biomass_palette}, "Manageable Bio Density")
# m.addLayer(manageable_soc_density.clip(y2y), {
#            "min": 0, "max": 450, "palette": biomass_palette}, "Manageable SOC Density")
# m.addLayer(vulnerable_bio_density.clip(y2y), {
#            "min": 0, "max": 450, "palette": biomass_palette}, "Vulnerable Bio Density")
# m.addLayer(vulnerable_soc_sl_density.clip(y2y), {
#            "min": 0, "max": 450, "palette": biomass_palette}, "Vulnerable SOC SL Density")
# m.addLayer(recoverable_soc_sl_density.clip(y2y), {
#            "min": 0, "max": 450, "palette": biomass_palette}, "Recoverable SOC SL Density")
# m.addLayer(irrecoverable_bio_density.clip(y2y), {
#            "min": 0, "max": 400, "palette": biomass_palette}, "Irrecoverable Bio Density")
# m.addLayer(irrecoverable_soc_density.clip(y2y), {
#            "min": 0, "max": 400, "palette": biomass_palette}, "Irrecoverable SOC Density")
# m.addLayer(irrecoverable_bio_density_top20, {
#            "min": 0, "max": 400, "palette": biomass_palette}, "Irrecoverable Bio Density Top 20%")
# m.addLayer(irrecoverable_combined_sl_density_top5.clip(y2y), {
#            "min": 0, "max": 400, "palette": biomass_palette}, "Irrecoverable Combined SL Density Top 5%")
# m.addLayer(irrecoverable_combined_sl_density_top1.clip(y2y), {
#            "min": 0, "max": 400, "palette": biomass_palette}, "Irrecoverable Combined SL Density Top 1%")
# m.addLayer(irrecoverable_combined_m_density_top5.clip(y2y), {
#            "min": 0, "max": 400, "palette": biomass_palette}, "Irrecoverable Combined M Density Top 5%")
# m.addLayer(irrecoverable_combined_m_density_top1.clip(y2y), {
#            "min": 0, "max": 400, "palette": biomass_palette}, "Irrecoverable Combined M Density Top 1%")
# m.addLayer(y2y_protected_areas, {}, "Protected Areas")
# m.addLayer(pixel_area_ha.updateMask(irrecoverable_bio_density_top20).clipToCollection(y2y_protected_areas), {}, 'Pixel Area Irr Bio Top 20 Protected')

# check masks
# m.addLayer(mask_check.clip(y2y), {}, "Mask Check")
# m.addLayer(temperate_forest_mask.clip(y2y), {}, "Temperate Forest Mask")
# m.addLayer(temperate_grass_shrub_mask.clip(y2y), {}, "Temperate Grasslands and Shrublands")
# m.addLayer(montane_grass_shrub_mask.clip(y2y), {}, "Montane Grasslands and Shrublands")
# m.addLayer(northern_grass_shrub_mask.clip(y2y), {}, "Northern Grasslands and Shrublands")
# m.addLayer(temperate_wetlands_mask.clip(y2y), {}, "Temperate Wetlands")
# m.addLayer(northern_wetlands_mask.clip(y2y), {}, "Northern Wetlands")
# m.addLayer(temperate_peatlands_mask.clip(y2y), {}, "Temperate Peatlands")
# m.addLayer(northern_peatlands_mask.clip(y2y), {}, "Northern Peatlands")
# m.addLayer(northern_barren_mask.clip(y2y), {}, "Northern Barren Lands")
# m.addLayer(desert_mask.clip(y2y), {}, "Desert Mask")
# m.addLayer(wdpa_mask.clip(y2y), {}, "WDPA Mask")

# add bivar raster layer
# m.addLayer(irrecoverable_combined_sl_netflux_bivar, {"min": 6, "max": 30,
#            "palette": bivariate_palette}, "Bivar")

# display the map
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Export rasters in native resolution

In [21]:
# export rasters to disk in native resolution
data_folder = './irrecoverable_data/'

# geemap.download_ee_image(manageable_bio_density,
#                          filename=data_folder + 'manageable_biomass_2021_t_ha.tif',
#                          region=y2y.geometry(),
#                          crs=irrecoverable_bio_density.projection().getInfo().get('crs'),
#                          crs_transform=irrecoverable_bio_density.projection().getInfo().get('transform'))

# geemap.download_ee_image(manageable_soc_density,
#                          filename=data_folder + 'manageable_soc_t_ha.tif',
#                          region=y2y.geometry(),
#                          crs=irrecoverable_soc_sl_density.projection().getInfo().get('crs'),
#                          crs_transform=irrecoverable_soc_sl_density.projection().getInfo().get('transform'))

# geemap.download_ee_image(vulnerable_bio_density,
#                          filename=data_folder + 'vulnerable_biomass_2021_t_ha.tif',
#                          region=y2y.geometry(),
#                          crs=irrecoverable_bio_density.projection().getInfo().get('crs'),
#                          crs_transform=irrecoverable_bio_density.projection().getInfo().get('transform'))

# geemap.download_ee_image(vulnerable_soc_sl_density,
#                          filename=data_folder + 'vulnerable_soc_sl_t_ha.tif',
#                          region=y2y.geometry(),
#                          crs=irrecoverable_soc_sl_density.projection().getInfo().get('crs'),
#                          crs_transform=irrecoverable_soc_sl_density.projection().getInfo().get('transform'))

# geemap.download_ee_image(vulnerable_soc_m_density,
#                          filename=data_folder + 'vulnerable_soc_m_t_ha.tif',
#                          region=y2y.geometry(),
#                          crs=irrecoverable_soc_sl_density.projection().getInfo().get('crs'),
#                          crs_transform=irrecoverable_soc_sl_density.projection().getInfo().get('transform'))

# geemap.download_ee_image(irrecoverable_bio_density,
#                          filename=data_folder + 'irrecoverable_biomass_2021_t_ha.tif',
#                          region=y2y.geometry(),
#                          crs=irrecoverable_bio_density.projection().getInfo().get('crs'),
#                          crs_transform=irrecoverable_bio_density.projection().getInfo().get('transform'))

# geemap.download_ee_image(irrecoverable_soc_sl_density,
#                          filename=data_folder + 'irrecoverable_sl_soc_t_ha.tif',
#                          region=y2y.geometry(),
#                          crs=irrecoverable_soc_sl_density.projection().getInfo().get('crs'),
#                          crs_transform=irrecoverable_soc_sl_density.projection().getInfo().get('transform'))

# geemap.download_ee_image(irrecoverable_soc_m_density,
#                          filename=data_folder + 'irrecoverable_m_soc_t_ha.tif',
#                          region=y2y.geometry(),
#                          crs=irrecoverable_soc_m_density.projection().getInfo().get('crs'),
#                          crs_transform=irrecoverable_soc_m_density.projection().getInfo().get('transform'))

# geemap.download_ee_image(irrecoverable_combined_sl_density,
#                          filename=data_folder + 'irrecoverable_sl_combined_t_ha.tif',
#                          region=y2y.geometry(),
#                          crs=irrecoverable_combined_sl_density.projection().getInfo().get('crs'),
#                          crs_transform=irrecoverable_combined_sl_density.projection().getInfo().get('transform'))

# geemap.download_ee_image(irrecoverable_combined_m_density,
#                          filename=data_folder + 'irrecoverable_m_combined_t_ha.tif',
#                          region=y2y.geometry(),
#                          crs=irrecoverable_combined_m_density.projection().getInfo().get('crs'),
#                          crs_transform=irrecoverable_combined_m_density.projection().getInfo().get('transform'))

# export masking 0 values for better quantile calculation
# geemap.download_ee_image(irrecoverable_bio_density.updateMask(irrecoverable_bio_density.gt(0)),
                        #  filename=data_folder + 'irrecoverable_biomass_2021_t_ha_gt0.tif',
                        #  region=y2y.geometry(),
                        #  crs=irrecoverable_bio_density.projection().getInfo().get('crs'),
                        #  crs_transform=irrecoverable_bio_density.projection().getInfo().get('transform'))

# geemap.download_ee_image(irrecoverable_soc_sl_density.updateMask(irrecoverable_soc_sl_density.gt(0)),
#                          filename=data_folder + 'irrecoverable_sl_soc_t_ha_gt0.tif',
#                          region=y2y.geometry(),
#                          crs=irrecoverable_soc_sl_density.projection().getInfo().get('crs'),
#                          crs_transform=irrecoverable_soc_sl_density.projection().getInfo().get('transform'))

# geemap.download_ee_image(irrecoverable_soc_m_density.updateMask(irrecoverable_soc_m_density.gt(0)),
#                          filename=data_folder + 'irrecoverable_m_soc_t_ha_gt0.tif',
#                          region=y2y.geometry(),
#                          crs=irrecoverable_soc_m_density.projection().getInfo().get('crs'),
#                          crs_transform=irrecoverable_soc_m_density.projection().getInfo().get('transform'))

# geemap.download_ee_image(irrecoverable_combined_sl_density.updateMask(irrecoverable_combined_sl_density.gt(0)),
#                          filename=data_folder + 'irrecoverable_sl_combined_t_ha_gt0.tif',
#                          region=y2y.geometry(),
#                          crs=irrecoverable_combined_sl_density.projection().getInfo().get('crs'),
#                          crs_transform=irrecoverable_combined_sl_density.projection().getInfo().get('transform'))

# geemap.download_ee_image(irrecoverable_combined_m_density.updateMask(irrecoverable_combined_m_density.gt(0)),
#                          filename=data_folder + 'irrecoverable_m_combined_t_ha_gt0.tif',
#                          region=y2y.geometry(),
#                          crs=irrecoverable_combined_m_density.projection().getInfo().get('crs'),
#                          crs_transform=irrecoverable_combined_m_density.projection().getInfo().get('transform'))



In [22]:
# export hotspot rasters to disk in native resolution
data_folder = './irrecoverable_data/'

# geemap.download_ee_image(irrecoverable_bio_density_top20,
#                          filename=data_folder + 'irrecoverable_bio_t_ha_top20.tif',
#                          region=y2y.geometry(),
#                          crs=irrecoverable_bio_density_top20.projection().getInfo().get('crs'),
#                          crs_transform=irrecoverable_bio_density_top20.projection().getInfo().get('transform'))

# geemap.download_ee_image(irrecoverable_soc_sl_density_top20,
#                          filename=data_folder + 'irrecoverable_soc_sl_t_ha_top20.tif',
#                          region=y2y.geometry(),
#                          crs=irrecoverable_soc_sl_density_top20.projection().getInfo().get('crs'),
#                          crs_transform=irrecoverable_soc_sl_density_top20.projection().getInfo().get('transform'))

# geemap.download_ee_image(irrecoverable_soc_m_density_top20,
#                          filename=data_folder + 'irrecoverable_soc_m_t_ha_top20.tif',
#                          region=y2y.geometry(),
#                          crs=irrecoverable_soc_m_density_top20.projection().getInfo().get('crs'),
#                          crs_transform=irrecoverable_soc_m_density_top20.projection().getInfo().get('transform'))

# geemap.download_ee_image(irrecoverable_combined_sl_density_top20,
#                          filename=data_folder + 'irrecoverable_sl_combined_t_ha_top20.tif',
#                          region=y2y.geometry(),
#                          crs=irrecoverable_combined_sl_density_top20.projection().getInfo().get('crs'),
#                          crs_transform=irrecoverable_combined_sl_density_top20.projection().getInfo().get('transform'))

# geemap.download_ee_image(irrecoverable_combined_m_density_top20,
#                          filename=data_folder + 'irrecoverable_m_combined_t_ha_top20.tif',
#                          region=y2y.geometry(),
#                          crs=irrecoverable_combined_m_density_top20.projection().getInfo().get('crs'),
#                          crs_transform=irrecoverable_combined_m_density_top20.projection().getInfo().get('transform'))

# geemap.download_ee_image(netflux_c_top20,
#                          filename=data_folder + 'netflux_c_t_ha_top20.tif',
#                          region=y2y.geometry(),
#                          crs=irrecoverable_bio_density.projection().getInfo().get('crs'),
#                          scale=2500)

# geemap.download_ee_image(irrecoverable_bio_netflux_top20,
#                          filename=data_folder + 'irrecoverable_bio_netflux_top20_2500.tif',
#                          region=y2y.geometry(),
#                          crs=irrecoverable_bio_density.projection().getInfo().get('crs'),
#                          scale=2500)

ee.batch.Export.image.toDrive(
    image=irrecoverable_bio_netflux_top20,
    fileNamePrefix='irrecoverable_bio_netflux_top20_2500',
    region=y2y.geometry(),
    scale=2500,
    crs=irrecoverable_bio_density.projection().getInfo().get('crs')
).start()

In [24]:
# export bivariate rasters in native resolution
data_folder = './irrecoverable_data/'

ee.batch.Export.image.toDrive(
    image=irrecoverable_bio_netflux_bivar,
    fileNamePrefix='irrecoverable_bio_netflux_bivar_1000',
    region=y2y.geometry(),
    scale=1000,
    crs=irrecoverable_bio_density.projection().getInfo().get('crs')
).start()

In [23]:
# export an ecosystems mask layer for figure
# create an ecosystems layer

# ecosystems = (
#     lc
#     .clip(y2y)
#     .where(temperate_forest_mask, 25)
#     .where(northern_forest_mask, 26)
#     .where(temperate_grass_shrub_mask, 27)
#     .where(montane_grass_shrub_mask, 28)
#     .where(northern_grass_shrub_mask, 29)
#     .where(temperate_wetlands_mask, 30)
#     .where(northern_wetlands_mask, 31)
#     .where(temperate_peatlands_mask, 32)
#     .where(northern_peatlands_mask, 33)
#     .where(temperate_barren_mask, 34)
#     .where(northern_barren_mask, 34)
#     .where(desert_mask, 35)
#     )

# # set values to 0 where they are between 1 and 19
# ecosystems = ecosystems.updateMask(ecosystems.gt(19))

# # export
# geemap.download_ee_image(ecosystems,
#                          filename=data_folder + 'ecosystems_classification_2500m.tif',
#                          region=y2y.geometry(),
#                          scale=2500)

## Define functions to calculate zonal statistics

In [24]:
# Define function to calculate zonal statistics
# specify image to use and reducer type (sum(), mean())

def calc_zonal_stats(feature, image_list, reducer_list):

    # loop through image and reducer list
    for image, reducer in zip(image_list, reducer_list):

        # calculate zonal stats across features
        stats = image.reduceRegion(
            reducer=reducer,
            geometry=feature.geometry(),
            crs=image.select(0).projection(),
            crsTransform=image.select(
                0).projection().getInfo().get('transform'),
            maxPixels=1e30
        )

        # add zonal stats to feature properties
        feature = feature.set(stats)

    return feature

# define function to calculate zonal stats over landcover types
# currently only calculates sum in the same way as using the function above


def calc_lc_stats(image, landcover, geo, scale):
    # loop through image band names
    for index, band in enumerate(image.bandNames().getInfo()):
        # select band and add lc
        img = image.select(band).addBands(landcover)

        # take sum of band grouped by lc
        stats = img.reduceRegion(
            reducer=ee.Reducer.sum().group(groupField=1, groupName='lc'),
            geometry=geo,
            scale=scale,
            crs=img.select(0).projection(),
            # crsTransform=img.select(
            #    0).projection().getInfo().get('transform'),
            maxPixels=1e30
        )

        # save dictionary in df
        if index == 0:
            df = pd.DataFrame(stats.getInfo().get('groups')
                              ).rename(columns={'sum': band})
        else:
            df = df.merge(pd.DataFrame(stats.getInfo().get(
                'groups')).rename(columns={'sum': band}), on='lc')

    # return df
    return df

# Define function to calculate zonal statistics over different biome/lc masks
# specify mask, feature, image to use and reducer type (sum(), mean())


def calc_zonal_stats_biome_lc_mask(mask, feature, image_list, reducer_list):

    # create empty dictionary
    stats = {}

    # loop through image and reducer list
    for image, reducer in zip(image_list, reducer_list):

        # calculate zonal stats across features
        stats_append = [
            image
            .updateMask(mask)
            .reduceRegion(
                reducer=reducer,
                geometry=feature.geometry(),
                crs=image.select(0).projection(),
                crsTransform=image.select(
                    0).projection().getInfo().get('transform'),
                maxPixels=1e30
            )
        ]

        # add to dictionary
        stats.update(stats_append[0].getInfo())

    # return dictionary
    return stats

# below functions not yet modified for this code
# not sure if we need them

# define function to calculate additional carbon stock values
# since pixel sizes vary instead of taking the mean of all density values from the rasters
# we are calculating the total carbon and dividing it by the area in ha


# def calc_addl_stats(df):
#     # calc total biomass
#     df['tob_t'] = df['agb_t'] + df['bgb_t']

#     # calc densities
#     df['agb_t_ha'] = df['agb_t'] / df['pixel_area_agb_extent_ha']
#     df['bgb_t_ha'] = df['bgb_t'] / df['pixel_area_agb_extent_ha']
#     df['tob_t_ha'] = df['tob_t'] / df['pixel_area_agb_extent_ha']
#     df['soc_t_ha'] = df['soc_t'] / df['pixel_area_soc_extent_ha']

#     # calc % of storage
#     df['perc_tob'] = (df['tob_t'] / df['tob_t'].sum()) * 100
#     df['perc_soc'] = (df['soc_t'] / df['soc_t'].sum()) * 100

#     # calc % of area
#     df['perc_tob_extent'] = (
#         df['pixel_area_agb_extent_ha'] / df['pixel_area_agb_extent_ha'].sum()) * 100
#     df['perc_soc_extent'] = (
#         df['pixel_area_soc_extent_ha'] / df['pixel_area_soc_extent_ha'].sum()) * 100
#     df['perc_area'] = (df['pixel_area_ha'] / df['pixel_area_ha'].sum()) * 100

#     # return df
#     return df

## Calculate zonal statistics

In [25]:
# define image and reducer list for zonal stats
img_list = [manageable_bio_stock,
            manageable_soc_stock,
            vulnerable_bio_stock,
            vulnerable_soc_sl_stock,
            vulnerable_soc_m_stock,
            irrecoverable_bio_stock,
            irrecoverable_soc_sl_stock,
            irrecoverable_soc_m_stock,
            irrecoverable_combined_sl_stock,
            irrecoverable_combined_m_stock]

redu_list = [ee.Reducer.sum(),
             ee.Reducer.sum(),
             ee.Reducer.sum(),
             ee.Reducer.sum(),
             ee.Reducer.sum(),
             ee.Reducer.sum(),
             ee.Reducer.sum(),
             ee.Reducer.sum(),
             ee.Reducer.sum(),
             ee.Reducer.sum(),
             ee.Reducer.sum()]

# define output folder for exports
out_folder = './outputs/'

In [27]:
# Compute zonal statistics across Y2Y region
y2y_carbon = y2y.map(lambda feature: calc_zonal_stats(
    feature, img_list, redu_list))

# convert to df
y2y_df = ee.data.computeFeatures({
    'expression': y2y_carbon,
    'fileFormat': 'PANDAS_DATAFRAME'
})

# # add density values
# y2y_df = calc_addl_stats(y2y_df)

# drop geometry, round values and export as csv
y2y_df.drop('geo', axis=1).round(2).to_csv(
    out_folder + 'y2y_irrecoverable_carbon_stocks.csv', index=False)

In [ ]:
# Compute zonal statistics across biomes
biome_carbon = y2y_biomes.map(lambda feature: calc_zonal_stats(
    feature, img_list, redu_list))

# convert to df
biome_df = ee.data.computeFeatures({
    'expression': biome_carbon,
    'fileFormat': 'PANDAS_DATAFRAME'
})

# # add density values
# biome_df = calc_addl_stats(biome_df)

# drop geometry, round values and export as csv
biome_df.drop('geo', axis=1).round(2).to_csv(
    out_folder + 'y2y_biome_irrecoverable_carbon_stocks.csv', index=False)

In [ ]:
# Compute zonal statistics across ecoregions
eco_carbon = y2y_ecoregions.map(lambda feature: calc_zonal_stats(
    feature, img_list, redu_list))

# convert to df
eco_df = ee.data.computeFeatures({
    'expression': eco_carbon,
    'fileFormat': 'PANDAS_DATAFRAME'
})

# # add density values
# eco_df = calc_addl_stats(eco_df)

# drop geometry, round values and export as csv
eco_df.drop('geo', axis=1).round(2).to_csv(
    out_folder + 'y2y_ecoregion_irrecoverable_carbon_stocks.csv', index=False)

In [ ]:
# Compute zonal statistics in Ross River IPCA
rra_carbon = rra.map(lambda feature: calc_zonal_stats(
    feature, img_list, redu_list))

# convert to df
rra_df = ee.data.computeFeatures({
    'expression': rra_carbon,
    'fileFormat': 'PANDAS_DATAFRAME'
})

# # add density values
# rra_df = calc_addl_stats(rra_df)

# drop geometry, round values and export as csv
rra_df.drop('geo', axis=1).round(2).to_csv(
    out_folder + 'y2y_rra_irrecoverable_carbon_stocks.csv', index=False)

In [23]:
# compute zonal statistics across landcover types
# manageable carbon
# bio first
lc_df = calc_lc_stats(manageable_bio_stock, lc, y2y.geometry(),
                      manageable_bio_stock.select(0).projection().nominalScale())

# merge with soc
lc_df = lc_df.merge(
    calc_lc_stats(manageable_soc_stock, lc, y2y.geometry(),
                  manageable_bio_stock.select(0).projection().nominalScale()),
    on=['lc']
)

# vulnerable carbon
# merge with bio
lc_df = lc_df.merge(
    calc_lc_stats(vulnerable_bio_stock, lc, y2y.geometry(),
                  manageable_bio_stock.select(0).projection().nominalScale()),
    on=['lc']
)

# merge with soc
lc_df = lc_df.merge(
    calc_lc_stats(vulnerable_soc_sl_stock, lc, y2y.geometry(),
                  manageable_bio_stock.select(0).projection().nominalScale()),
    on=['lc']
)

lc_df = lc_df.merge(
    calc_lc_stats(vulnerable_soc_m_stock, lc, y2y.geometry(),
                  manageable_bio_stock.select(0).projection().nominalScale()),
    on=['lc']
)

# irrecoverable carbon
# merge with bio
lc_df = lc_df.merge(
    calc_lc_stats(irrecoverable_bio_stock, lc, y2y.geometry(),
                  manageable_bio_stock.select(0).projection().nominalScale()),
    on=['lc']
)

# merge with soc
lc_df = lc_df.merge(
    calc_lc_stats(irrecoverable_soc_sl_stock, lc, y2y.geometry(),
                  manageable_bio_stock.select(0).projection().nominalScale()),
    on=['lc']
)

# merge with soc
lc_df = lc_df.merge(
    calc_lc_stats(irrecoverable_soc_m_stock, lc, y2y.geometry(),
                  manageable_bio_stock.select(0).projection().nominalScale()),
    on=['lc']
)

# # drop duplicate pixel_area_ha_column
# lc_df = lc_df.drop(columns=['pixel_area_ha_y']).rename(
#     columns={'pixel_area_ha_x': 'pixel_area_ha'})

# # add density values
# lc_df = calc_addl_stats(lc_df)

# set landcover mapping key
landcover_mapping = {
    1: "Temperate or sub-polar needleleaf forest",
    2: "Sub-polar taiga needleleaf forest",
    3: "Tropical or sub-tropical broadleaf evergreen forest",
    4: "Tropical or sub-tropical broadleaf deciduous forest",
    5: "Temperate or sub-polar broadleaf deciduous forest",
    6: "Mixed forest",
    7: "Tropical or sub-tropical shrubland",
    8: "Temperate or sub-polar shrubland",
    9: "Tropical or sub-tropical grassland",
    10: "Temperate or sub-polar grassland",
    11: "Sub-polar or polar shrubland-lichen-moss",
    12: "Sub-polar or polar grassland-lichen-moss",
    13: "Sub-polar or polar barren-lichen-moss",
    14: "Wetland",
    15: "Cropland",
    16: "Barren land",
    17: "Urban and built-up",
    18: "Water",
    19: "Snow and ice"
}

# map landcover names
lc_df['lc'] = lc_df['lc'].map(landcover_mapping)
lc_df

# round values and export as csv
lc_df.round(2).to_csv(
    out_folder + 'y2y_lc_irrecoverable_carbon_stocks.csv', index=False)

In [ ]:
# compute zonal statistics across biome/lc classifications
# create dictionary with masks
mask_dict = {
    'Temperate Forest':temperate_forest_mask, 
    'Northern Forest':northern_forest_mask, 
    'Temperate Grass Shrub':temperate_grass_shrub_mask,
    'Montane Grass Shrub':montane_grass_shrub_mask, 
    'Northern Grass Shrub':northern_grass_shrub_mask, 
    'Temperate Wetlands':temperate_wetlands_mask,
    'Northern Wetlands':northern_wetlands_mask, 
    'Temperate Peatlands':temperate_peatlands_mask, 
    'Northern Peatlands':northern_peatlands_mask,
    'Temperate Barren':temperate_barren_mask, 
    'Northern Barren':northern_barren_mask
    }

# create empty dataframe
biome_lc_df = pd.DataFrame()

# loop through biome/lc masks
for name, mask in mask_dict.items():
    df_hold = calc_zonal_stats_biome_lc_mask(mask, y2y, img_list, redu_list)
    biome_lc_df = pd.concat([biome_lc_df, pd.DataFrame(df_hold, index=[name])])

# round values and export as csv
biome_lc_df.round(2).to_csv(
    out_folder + 'y2y_biome_lc_masks_irrecoverable_carbon_stocks.csv', index=True)

In [32]:
# compute zonal statistics across protected areas
# combine feature collections and dissolve into single polygon
y2y_protected_areas = y2y_wdpa.merge(y2y_oecm)

# clip zonal stat images to collection
# since protected areas is too complex
irr_bio_stock_prot = irrecoverable_bio_stock.clipToCollection(y2y_protected_areas)
irr_soc_sl_stock_prot = irrecoverable_soc_sl_stock.clipToCollection(y2y_protected_areas)
irr_soc_m_stock_prot = irrecoverable_soc_m_stock.clipToCollection(y2y_protected_areas)

# define image and reducer list for zonal stats
img_list_prot = [irr_bio_stock_prot, irr_soc_sl_stock_prot, irr_soc_m_stock_prot]
redu_list_prot = [ee.Reducer.sum(), ee.Reducer.sum(), ee.Reducer.sum()]

protected_areas_carbon = y2y.map(lambda feature: calc_zonal_stats(
    feature, img_list_prot, redu_list_prot))

# convert to df
protected_areas_df = ee.data.computeFeatures({
    'expression': protected_areas_carbon,
    'fileFormat': 'PANDAS_DATAFRAME'
})

# drop geometry column
protected_areas_df = protected_areas_df.drop(
    ['geo'], axis=1)

# create row for non protected areas
non_protected_row = pd.DataFrame([{'irrecoverable_biomass_t': (y2y_df['irrecoverable_biomass_t'] - protected_areas_df['irrecoverable_biomass_t'].sum()).values[0],
                                   'irrecoverable_soc_m_t': (y2y_df['irrecoverable_soc_m_t'] - protected_areas_df['irrecoverable_soc_m_t'].sum()).values[0],
                                   'irrecoverable_soc_sl_t': (y2y_df['irrecoverable_soc_sl_t'] - protected_areas_df['irrecoverable_soc_sl_t'].sum()).values[0]}])

# add non protected row to df
protected_areas_df = pd.concat(
    [protected_areas_df, non_protected_row], ignore_index=True)

# # add density values
# protected_areas_df = calc_addl_stats(protected_areas_df)

# round values and export as csv
protected_areas_df.round(2).to_csv(
    out_folder + 'y2y_protected_areas_irrecoverable_carbon_stocks.csv', index=False)

In [55]:
# calculate irrecoverable carbon hotspots in protected areas
# merge feature collections of protected areas
y2y_protected_areas = y2y_wdpa.merge(y2y_oecm)

# calculate total area of irr biomass hotspots
irr_bio_area = (
    pixel_area_ha
    .updateMask(irrecoverable_bio_density_top20)
    .reduceRegion(reducer=ee.Reducer.sum(),
                  geometry=y2y.geometry(),
                  scale=irrecoverable_bio_density_top20.projection().nominalScale(),
                  maxPixels=1e30)
)

irr_bio_prot = (
    pixel_area_ha
    .updateMask(irrecoverable_bio_density_top20)
    .clipToCollection(y2y_protected_areas)
    .reduceRegion(reducer=ee.Reducer.sum(),
                  geometry=y2y.geometry(),
                  scale=irrecoverable_bio_density_top20.projection().nominalScale(),
                  maxPixels=1e30)
)

print('Irrecoverable Biomass Total Area:', irr_bio_area.get('area').getInfo(),
      '\nIrrecoverable Biomass Protected Area:', irr_bio_prot.get('area').getInfo(),
      '\nIrrecoverable Biomass Non-Protected Area', irr_bio_area.get('area').getInfo() - irr_bio_prot.get('area').getInfo())

Irrecoverable Biomass Total Area: 9747503.363698872 
Irrecoverable Biomass Protected Area: 1284443.559094427 
Irrecoverable Biomass Non-Protected Area 8463059.804604445
